In [85]:
#Import Custom Tools
import ldasort.ldasort as ldasort

## Import Standard Data Processing Tools
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer

#import ML tools
import tensorflow as tf
from tensorflow import keras

from importlib import reload
reload(ldasort)

<module 'ldasort.ldasort' from '/home/kbahnsen/Development/Marissa/ldasort/ldasort/ldasort.py'>

In [86]:
#LOAD Fashion MNIST
from tensorflow.keras.datasets import fashion_mnist

data_set = "FashionMNIST" 

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

#normalize
x_train_normalized = (x_train - np.mean(x_train))/np.std(x_train) 
x_test_normalized = (x_test - np.mean(x_train))/np.std(x_train)

#flatten and reshape
x_train_normalized = x_train_normalized.reshape(60000,28*28)
x_test_normalized = x_test_normalized.reshape(10000,28*28)
y_train=np.array(y_train)


In [87]:
# Find weights and biases using LDA sorting algorithm. 
# Comment out if weights already present. 

weights, biases = ldasort.ldasort(x_train_normalized, y_train, verbose=True)

Label 0 sorted
Label 1 sorted
Label 2 sorted
Label 3 sorted
Label 4 sorted
Label 5 sorted
Label 6 sorted
Label 7 sorted
Label 8 sorted
Label 9 sorted


In [88]:
print(len(weights))

28


In [80]:
# Save weights and biases for use later. 

np.savez("Sorted_Weights_{}".format(data_set), weights=weights, biases=biases)

In [81]:
# Load to run trials 

file = np.load("Sorted_Weights_{}.npz".format(data_set))

weights = file["weights"]
biases = file["biases"]

In [82]:
### create loop for comparing across batch sizes, learning rates

learning_rates = [.001, .005, .01]
batch_sizes = [25, 100, 500]

#small values for quick sample run. For full length runs, use n_trials=20, n_epochs=100
n_trials = 5
n_epochs = 5

t0=timer()

for batch_size in batch_sizes:
    for lr in learning_rates:

        temp_hists=[]
        val_hists=[]

        for trial in range(n_trials):

            #set up LDA-initialized model
            
            model=ldasort.setup(intermediate_neurons=len(weights),input_shape=784, output_shape=10)

            model.compile(
                    optimizer=keras.optimizers.SGD(learning_rate=lr),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'],
                    weighted_metrics=['accuracy']
                    )

            model.layers[0].set_weights([weights.T,biases])
        

            history=model.fit(x_train_normalized, 
                      y_train, 
                      epochs=n_epochs, 
                      batch_size=batch_size,
                      validation_data = (x_test_normalized, y_test),
                      verbose=False)

            temp_hists.append(history.history['accuracy'])
            val_hists.append(history.history["val_accuracy"])


            keras.backend.clear_session()

        lda_hist = temp_hists
        val_lda_hist=val_hists


        #######################################
        #Run Trials with Random Initialization#
        #######################################
        temp_hists=[]
        val_hists=[]

        for trial in range(n_trials):

            #set up randomly-initialized model
            
            model=ldasort.setup(intermediate_neurons=len(weights),input_shape=784, output_shape=10)

            model.compile(
                    optimizer=keras.optimizers.SGD(learning_rate=lr),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'],
                    weighted_metrics=['accuracy']
                    )
            
            #do not set weights
            #model.layers[0].set_weights([weights.T,biases])
            


            history=model.fit(x_train_normalized, 
                      y_train, 
                      epochs=n_epochs, 
                      batch_size=batch_size,
                      validation_data = (x_test_normalized, y_test),
                      verbose=False)

            temp_hists.append(history.history['accuracy'])
            val_hists.append(history.history["val_accuracy"])
            

            keras.backend.clear_session()

        rand_hist = temp_hists
        val_rand_hist=val_hists
        

        np.savez("{}_sigmoid_lr{}_bat{}".format(data_set, lr,batch_size), 
                 lda_acc=lda_hist, 
                 val_lda_acc=val_lda_hist,
                 rand_acc = rand_hist, 
                 val_rand_acc = val_rand_hist,
                 n_components=len(weights), 
                num_trials=n_trials,
                n_epochs = n_epochs)
        
        
        print("Trial with Batch Size {} Learning Rate {} Done, time elapsed {} minutes".format(
            batch_size, lr, np.round((timer()-t0)/60),3))



Trial with Batch Size 25 Learning Rate 0.001 Done, time elapsed 2.0 minutes
Trial with Batch Size 25 Learning Rate 0.005 Done, time elapsed 4.0 minutes
Trial with Batch Size 25 Learning Rate 0.01 Done, time elapsed 6.0 minutes
Trial with Batch Size 100 Learning Rate 0.001 Done, time elapsed 7.0 minutes
Trial with Batch Size 100 Learning Rate 0.005 Done, time elapsed 7.0 minutes


KeyboardInterrupt: 

In [84]:
# Construct plots. 

plt.rcParams.update(plt.rcParamsDefault)

data = dict()

for batch_size in batch_sizes:
    for lr in learning_rates:
        data[(batch_size, lr)]=np.load("{}_sigmoid_lr{}_bat{}.npz".format(data_set, lr, batch_size))
        
        fig,ax=plt.subplots()
        

        for randplot,ldaplot in zip(data[(batch_size, lr)]["val_rand_acc"],data[(batch_size, lr)]["val_lda_acc"]): #j is random h is preset 
            ax.plot(ldaplot,c='red',alpha=.1)
            ax.plot(randplot,c='blue',alpha=.1)


        #ax.plot([threshold]*nepochs,c='k',alpha=.2)
        ax.set_ylim((0,1))
        ax.set_title("FashionMNIST Sigmoid \n lr {}, batch size {}".format(lr, batch_size))      
        ax.legend(["LDA Sorted","Random"])
        plt.savefig("imgs/fashionmnist_lr{}_bat{}.png".format(lr,batch_size))
        plt.close()


0.01